In [1]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
import time
from pathlib import Path
from datetime import datetime
import SimpleITK as sitk
from monai.utils import set_determinism
import math
import torch
from torch.utils.data import random_split, DataLoader
import monai
import gdown
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset,Dataset,PersistentDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract

sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()

from datetime import datetime
import os
import tempfile
from glob import glob

from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import torch.nn as nn
import torch.nn.functional as F


sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()
import preprocessing.transformsForMain as transformsForMain
import preprocessing.ManageMetadata as manageMetaData
import dataManag.utils.dataUtils as dataUtils
import multiprocessing
from model.unets import UNet
import model.DataModule
import model.LigtningModel as LigtningModel


comet_logger = CometLogger(
    api_key="yB0irIjdk9t7gbpTlSUPnXBd4",
    #workspace="OPI", # Optional
    project_name="picai_base_3Channels", # Optional
    #experiment_name="baseline" # Optional
)
df= manageMetaData.load_df_only_full()


data = model.DataModule.PiCaiDataModule(
    df= df,
    batch_size=3,#TODO(batc size determined by lightning)
    trainSizePercent=0.7,
    num_workers=os.cpu_count(),
    drop_last=False,#True,
    cache_dir="/home/sliceruser/preprocess/monai_persistent_Dataset",
    t2w_name="t2w_med_spac",
    adc_name="registered_adc_med_spac",
    hbv_name="registered_hbv_med_spac",
    label_name="label_med_spac",
    maxSize=manageMetaData.getMaxSize("t2w_med_spac",df)
)
data.prepare_data()
data.setup()

# definition described in model folder
unet =UNet()

model = LigtningModel.Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True,onehot_y=True, ), # Our seg labels are single channel images indicating class index, rather than one-hot
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)
trainer = pl.Trainer(
    #accelerato="cpu", #TODO(remove)
    max_epochs=3,
    #gpus=1,
    precision=16, #TODO(unhash)
    callbacks=[early_stopping],#TODO(unhash)
    logger=comet_logger,
    accelerator='auto',
    devices='auto',
    default_root_dir= "/home/sliceruser/lightning_logs",
)
trainer.logger._default_hp_metric = False


start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)
#experiment.end()


aaa


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train data set: 154
Test data set: 33
Valid data set: 33
Training started at 2022-06-21 05:24:11.909102
Train data set: 154
Test data set: 33
Valid data set: 33


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
#experiment.end()